In [6]:
#import dependencies
import torch
from PIL import Image
from torch import nn,save,load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
#use gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#wandb
import wandb
wandb.login()

True

In [ ]:
# Get data
train = datasets.MNIST(root="./data", download=True, train=True, transform=ToTensor())
dataset = DataLoader(train,32) #batches of 32 images

In [ ]:
input_size = 784 # flattened 28x28 images
hidden_size = 500 
num_classes = 10 # numbers from 0 to 9 
num_epochs = 20
batch_size = 100
learning_rate = 0.001

In [ ]:
#Image Classifier Neural Network

class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier,self).__init__()
        self.model=nn.Sequential(
        nn.Conv2d(1,32,(3,3)),
        nn.ReLU(),
        nn.Conv2d(32,64,(3,3)),
        nn.ReLU(),        
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),        
        nn.Flatten(),
        #input shape:64 final output from last conv2d * (28-6) since each convlution layer shaves off 2 rows/columns
        #output shape is the number of classes (10, cause 10 digits)
        nn.Linear(64*(28-6)*(28-6),10) 
        )
        
    def forward(self,x):
        return self.model(x)

In [ ]:
#Instance of the NN,loss,optimizer

clf = ImageClassifier().to(device)
opt= Adam(clf.parameters(),lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Training flow 

if __name__ == "__main__":
    for epoch in num_epochs: #train for 10 epochs
        for batch in dataset: #looping through the dataset
            X,y=batch
            X,y= X.to(device), y.to(device)
            yhat=clf(X)
            loss=loss_fn(yhat,y)
            
            #Apply backprop
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            #print(f'epoch {epoch+1}, loss is {loss.item()}')

In [ ]:
with open('model_state.pt','wb') as f:
    save(clf.state_dict(),f)

In [ ]:
if __name__ == "__main__":
    with open('model_state.pt','rb') as f:
        clf.load_state_dict(load(f))

img = Image.open('img_1.jpg')
img_tensor= ToTensor()(img).unsqueeze(0).to(device)
print(torch.argmax(clf(img_tensor)))